## Convert all netCDF NCAR timestep files to Zarr 512 arrays, with Grouped Velocity components, with (64,64,64) chunk size, round-robined across FileDB nodes (spatially using Z-order)

<font color="red">Old Dask version gives this error https://github.com/dask/distributed/issues/3955</font>

<font color='orange'>Note: Careful when Setting Dask `local_directory` to remote server (e.g. Temporary) will HUGELY slow down functions</font>

<font color='cyan'>Parallel version needs Large job</font>

<font color = 'gold'>TODO fix MemoryError: Unable to allocate 32.0 GiB for an array with shape (2048, 2048, 2048) and data type float32 when looping over multiple timesteps</font>

In [1]:
desired_cube_side = 512
chunk_size = 64
raw_ncar_folder_path = '/home/idies/workspace/turb/data02_02/ncar-high-rate-fixed-dt'
use_dask = True
dest_folder_name = "sabl2048b" # B is the high-rate data
write_type = "prod" # or "back" for backup

n_dask_workers = 4 # For Dask rechunking
num_threads = 34  # For writing to FileDB
dask_local_dir = '/home/idies/workspace/turb/data02_02'


timestep_nr = 0
# timestep_range = range(1) # This doesn't work with MemoryError: Unable to allocate 32.0 GiB for an array with shape (2048, 2048, 2048) and data type float32

In [2]:
%cd /home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code/zarr_writing

/home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code/zarr_writing


In [3]:
import xarray as xr
from utils import write_tools
import os
import threading
import queue
import numpy as np

### Get target Folder list

In [4]:
folders=write_tools.list_fileDB_folders()

# Avoiding 7-2 and 9-2 - they're too full as of May 2023
folders.remove("/home/idies/workspace/turb/data09_02/zarr/")
folders.remove("/home/idies/workspace/turb/data07_02/zarr/")

for i in range(len(folders)):
    folders[i] += dest_folder_name + "_" + str(i + 1).zfill(2) + "_" + write_type + "/"

# folders[:5]

# Create top-level dirs

# for folder_path in folders:
#     os.makedirs(folder_path, exist_ok=False)

<font color="orange">Don't delete the CD cell!</font>

In [5]:
%cd /home/idies/workspace/turb/data02_02/ncar-high-rate-fixed-dt

/home/idies/workspace/turb/data02_02/ncar-high-rate-fixed-dt


In [ ]:
# for timestep_nr in timestep_range:
data_xr = xr.open_dataset(raw_ncar_folder_path + "/jhd." + str(timestep_nr).zfill(3) + ".nc")

# Group 3 velocity components together
# This fails with Dask bcs. of write permission error on SciServer Job
# Never use dask with remote location on this!!
merged_velocity = write_tools.merge_velocities(data_xr, dask_local_dir=dask_local_dir
                                               , chunk_size_base=chunk_size, use_dask=True, n_dask_workers=n_dask_workers)


# Unabbreviate 'e', 'p', 't' variable names
merged_velocity = merged_velocity.rename({'e': 'energy', 't': 'temperature', 'p': 'pressure'})


dims = [dim for dim in data_xr.dims]
dims.reverse() # use (nnz, nny, nnx) instead of (nnx, nny, nnz)

# Split 2048^3 into smaller 512^3 arrays
smaller_groups, range_list = write_tools.split_zarr_group(merged_velocity, desired_cube_side, dims)

# Given up in favor of Ryan's node coloring technique
#     z_order = write_tools.morton_order_cube(cube_side=4)
node_assignments = write_tools.node_assignment(cube_side=4)


cubes = smaller_groups


print('Done preparing data. Starting to verify...')

In [ ]:
sorted_morton_list = [] # Sorting by Morton code to be consistent with Isotropic8192
array_cube_side=data_xr['e'].shape[0]


for i in range(len(range_list)):            
    min_coord = [a[0] for a in range_list[i]]
    max_coord = [a[1] - 1 for a in range_list[i]]
            
    sorted_morton_list.append((write_tools.morton_pack(array_cube_side, min_coord[0], min_coord[1], min_coord[2]), write_tools.morton_pack(array_cube_side, max_coord[0], max_coord[1], max_coord[2])))
        # (write_tools.morton_pack(array_cube_side, min_coord[2], min_coord[1], min_coord[0]), write_tools.morton_pack(array_cube_side, max_coord[2], max_coord[1], max_coord[0]))

        
sorted_morton_list = sorted(sorted_morton_list)
sorted_morton_list[:5]

# Here, instead of writing to disk, read from disk and compare

In [12]:
def verify_write(q):
    while True:
        try:
            original_512, zarr_512_path = q.get(timeout=10)  # Adjust timeout as necessary

            print(f"Reading Zarr from {zarr_512_path}...")
            zarr_512 = xr.open_zarr(zarr_512_path)

            # Compare attributes
            assert original_512.attrs == zarr_512.attrs

            # Compare each variable's data
            for var in original_512.data_vars:
                assert np.array_equal(original_512[var].values, zarr_512[var].values)
                assert original_512[var].attrs == zarr_512[var].attrs  # Compare attributes of the variable
            
            print(f"{zarr_512_path} CORRECT!")
        except queue.Empty:
            break
        except AssertionError:
            print(f"Data mismatch found for {zarr_512_path}.")
        finally:
            q.task_done()

            
def flatten_3d_list(lst_3d):
    return [element for sublist_2d in lst_3d for sublist_1d in sublist_2d for element in sublist_1d]

def search_dict_by_value(dictionary, value):
    for key, val in dictionary.items():
        if val == value:
            return key
    return None  # Value not found in the dictionary

In [13]:
flattened_node_assgn = flatten_3d_list(node_assignments)

mike_dict = {}
for i in range(len(range_list)):            
    min_coord = [a[0] for a in range_list[i]]
    max_coord = [a[1] - 1 for a in range_list[i]]
            
    mike_dict[dest_folder_name + str(i + 1).zfill(2)] = sorted_morton_list[i]

In [15]:
cubes = flatten_3d_list(cubes)

In [16]:
q = queue.Queue()


# Populate the queue with tasks
for i in range(len(range_list)):
#     for j in range(4):
#         for k in range(4):
    min_coord = [a[0] for a in range_list[i]]
    max_coord = [a[1] - 1 for a in range_list[i]]
    
    morton = (write_tools.morton_pack(array_cube_side, min_coord[2], min_coord[1], min_coord[0]), write_tools.morton_pack(array_cube_side, max_coord[2], max_coord[1], max_coord[0]))
    
    chunk_name = search_dict_by_value(mike_dict, morton)
    
    idx = int(chunk_name[-2:].lstrip('0'))
    
    filedb_index = flattened_node_assgn[idx - 1] - 1
    
    destination = os.path.join(folders[filedb_index], dest_folder_name + str(idx).zfill(2) + "_" + str(timestep_nr).zfill(3) + ".zarr")
    
    current_array = cubes[i]
            
    q.put((current_array, destination))

In [ ]:
# Create threads and start them

threads = []
for _ in range(num_threads):
    t = threading.Thread(target=verify_write, args=(q,))
    t.start()
    threads.append(t)

# Wait for all tasks to be processed
q.join()

# Wait for all threads to finish
for t in threads:
    t.join()